In [1]:
'''
Created on Jun 24, 2018

@author: Mark Holton
'''

import sys


import pandas as pd
import numpy as np


print("start")

start


Comments are for code below
Reads a text file in and converts it to a dataframe. Due to the large size of the file a smaller file is created with hard coded columns for testing.

In [2]:
#read the file in and
textfile = "./ag_map_with_alpha.txt"
originalDF=pd.read_csv(textfile,
                        sep = "\t")
masterDF = originalDF.copy()
masterDF_short=masterDF.loc[0:20,['#SampleID',  
                                            'age_years',    
                                            'antibiotic_history',    
                                            'bmi_cat',  
                                            'depression_bipolar_schizophrenia', 
                                            'mental_illness_type_depression', 
                                            'smoking_frequency']].set_index('#SampleID', drop = False)
masterDF = masterDF_short.copy()  


/home/markholton/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Reads in user submited txt files and ask for user input to determine if data in the exclude file is to be filtered out or used to exclude everything that does not match it.

In [3]:
user_input_exclude = 'exclude' # input("Filename for file with column and data to exclude: ")
user_input_exclude_in_match = False # input('Is the "exclude" data to be excluded or kept (T for excluded and F for kept') == 'T'
user_input_control = 'control' # input("Filename for file with column and data to determine control: ")
user_input_experiment = 'experiment' # input("Filename for file with column and data to determine experimental: ")
user_input_match = 'match' # input("Filename for file with columns, type of value, and offset if needed to match samples: ")


user_input_exclude = './%s.txt'%(user_input_exclude)
user_input_control = './%s.txt'%(user_input_control)
user_input_experiment = './%s.txt'%(user_input_experiment )
user_input_match= './%s.txt'%(user_input_match)


'''print(user_input_exclude)
print(user_input_exclude_in_match)
print(user_input_control)
print(user_input_experiment)
print(user_input_match)'''

'print(user_input_exclude)\nprint(user_input_exclude_in_match)\nprint(user_input_control)\nprint(user_input_experiment)\nprint(user_input_match)'

Turns the user files into dataframes

In [4]:
user_input_excludeDF = pd.read_csv(user_input_exclude, sep = "\t")
user_input_controlDF = pd.read_csv(user_input_control, sep = "\t")
user_input_experimentDF = pd.read_csv(user_input_experiment, sep = "\t")
user_input_matchDF = pd.read_csv(user_input_match, sep = "\t")



Filters out unwanted rows based on values in chosen columns

In [5]:
inputDF = user_input_excludeDF.copy()
exclude_if_match = user_input_exclude_in_match
    
rows_num , columns_num = inputDF.shape
print('rows  = %s, columns = %s'%(rows_num, columns_num) )
r_counter = 0
c_counter = 0

while c_counter < columns_num:
    column_name = inputDF.columns[c_counter]
    print(column_name)
    masterDF = masterDF[
                masterDF[column_name].isin( inputDF[column_name].tolist() ) 
                == 
                ( not exclude_if_match ) ]
    c_counter = c_counter + 1
    
#masterDF = masterDF.reset_index(drop=True)
excludeDF = masterDF.copy()    


rows  = 1, columns = 1
antibiotic_history


Lables the samples based on if they are control or not

In [6]:
masterDF = excludeDF.copy()

inputDF = user_input_controlDF.copy()
    
rows_num , columns_num = inputDF.shape
print('rows  = %s, columns = %s'%(rows_num, columns_num) )
r_counter = 0
c_counter = 0
masterDF['case_control'] = False
while c_counter < columns_num:
    column_name = inputDF.columns[c_counter]
    masterDF['case_control'] = np.where( masterDF['case_control'].isin([True]) | masterDF[column_name].isin( inputDF[column_name].tolist() ).isin([True]), True, False)
    
    c_counter = c_counter + 1

masterDF['case_control'] = masterDF['case_control'].replace(True, 'control')   
masterDF['case_control'] = masterDF['case_control'].replace(False, 'Unspecified' )

controlDF = masterDF.copy()


rows  = 1, columns = 2


Lables the samples based on if they are cases or not

In [7]:
masterDF = excludeDF.copy()

inputDF = user_input_experimentDF.copy()
    
rows_num , columns_num = inputDF.shape
print('rows  = %s, columns = %s'%(rows_num, columns_num) )
r_counter = 0
c_counter = 0


masterDF['case_control'] = False
while c_counter < columns_num:
    column_name = inputDF.columns[c_counter]
    masterDF['case_control'] = np.where( masterDF['case_control'].isin([True]) | masterDF[column_name].isin( inputDF[column_name].tolist() ).isin([True]), True, False)
    
    c_counter = c_counter + 1

masterDF['case_control'] = masterDF['case_control'].replace(True, 'case')   
masterDF['case_control'] = masterDF['case_control'].replace(False, 'Unspecified')


    

experDF = masterDF.copy()


rows  = 3, columns = 2


Combines the two case_control columns into one column using pandas.DataFrame.where then makes this the master dataframe's case_control column

In [8]:
masterDF['case_control'] = controlDF['case_control'].where( 
                                                            controlDF['case_control'].isin(['control']), 
                                                            experDF['case_control'])

case_controlDF = masterDF.copy()

Filters out data with match columns left unspecified

In [9]:
masterDF = case_controlDF.copy()

inputDF = user_input_matchDF.copy()
    
rows_num , columns_num = inputDF.shape
print('rows  = %s, columns = %s'%(rows_num, columns_num) )
r_counter = 0
c_counter = 0
while c_counter < columns_num:
        column_name = inputDF.columns[c_counter]
        masterDF = masterDF.where(masterDF[column_name] != 'Unspecified')
        c_counter = c_counter + 1
masterDF = masterDF.dropna()
#masterDF = masterDF.reset_index(drop=True)

        
match_filteredDF = masterDF.copy()
#match_filteredDF

rows  = 2, columns = 3


In [10]:
def match_to_master(master_frame, match_frame, case_index, case_row):
    print(match_frame)
    for index, row in match_frame.iterrows():
        print(index)
        master_frame.at[index, 'matched to'] = case_row['#SampleID']
        master_frame.at[case_index, 'matched to'] = 'best match place holder'
    
    
    return master_frame

Matches samples labled experiment to control samples

In [11]:
masterDF = match_filteredDF.copy()

inputDF = user_input_matchDF.copy()
    
rows_num , columns_num = inputDF.shape
print('rows  = %s, columns = %s'%(rows_num, columns_num) )
r_counter = 0
c_counter = 0
case_for_matchDF = masterDF[masterDF['case_control'].isin(['case'])]
masterDF['matched to'] = 'none'
matchDF = masterDF.copy()
matchDF = matchDF[matchDF['case_control'].isin(['control'])]
for index, row in case_for_matchDF.iterrows():
    #print('index = %s\nrow = %s' %( index, row))
    while c_counter < columns_num:
        column_name = inputDF.columns[c_counter]
        print(column_name)
        datatype = inputDF.iloc[0,c_counter]
        if datatype == 'string':
            matchDF = matchDF[matchDF[column_name].isin([ row[column_name] ]) ]
            print('string')
        if datatype == 'number':
            matchDF = matchDF[
                                ( pd.to_numeric(matchDF[column_name]) >= ( int(float(row[column_name])) - int(float(inputDF.iloc[1,c_counter])) ) ) 
                                &
                                ( pd.to_numeric(matchDF[column_name]) <= ( int(float(row[column_name])) + int(float(inputDF.iloc[1,c_counter])) ) )
                               ] 
            print('num')
        c_counter = c_counter + 1
    masterDF = match_to_master(masterDF, matchDF, index, row)

#matchDF = masterDF.copy()
masterDF

rows  = 2, columns = 3
bmi_cat
string
age_years
num
smoking_frequency
string
                       #SampleID age_years  \
#SampleID                                    
10317.000030322  10317.000030322      43.0   

                                             antibiotic_history bmi_cat  \
#SampleID                                                                 
10317.000030322  I have not taken antibiotics in the past year.  Normal   

                depression_bipolar_schizophrenia  \
#SampleID                                          
10317.000030322     I do not have this condition   

                mental_illness_type_depression smoking_frequency case_control  \
#SampleID                                                                       
10317.000030322                    Unspecified             Never      control   

                matched to  
#SampleID                   
10317.000030322       none  
10317.000030322


,#SampleID,age_years,antibiotic_history,bmi_cat,depression_bipolar_schizophrenia,mental_illness_type_depression,smoking_frequency,case_control,matched to
#SampleID,,,,,,,,,
10317.000049761,10317.000049761,52,I have not taken antibiotics in the past year.,Overweight,Unspecified,Unspecified,Never,Unspecified,none
10317.000030322,10317.000030322,43.0,I have not taken antibiotics in the past year.,Normal,I do not have this condition,Unspecified,Never,control,10317.000029448
10317.000028857,10317.000028857,56.0,I have not taken antibiotics in the past year.,Overweight,I do not have this condition,Unspecified,Never,control,none
10317.000038189,10317.000038189,39.0,I have not taken antibiotics in the past year.,Overweight,Unspecified,Unspecified,Never,Unspecified,none
10317.000036487,10317.000036487,46.0,I have not taken antibiotics in the past year.,Obese,I do not have this condition,Unspecified,Never,control,none
10317.000030166,10317.000030166,58.0,I have not taken antibiotics in the past year.,Overweight,I do not have this condition,Unspecified,Never,control,none
10317.000023295,10317.000023295,56.0,I have not taken antibiotics in the past year.,Normal,I do not have this condition,Unspecified,Never,control,none
10317.000001589,10317.000001589,58.0,I have not taken antibiotics in the past year.,Normal,Unspecified,Unspecified,Never,Unspecified,none
10317.000030991,10317.000030991,63.0,I have not taken antibiotics in the past year.,Overweight,I do not have this condition,Unspecified,Never,control,none
